## Import relevant packages

In [ ]:
import pandas as pd
import os

from milton import *
from milton.batch import *
from milton.data_source import UKB_DATA_LOCATION
from milton.processing import GenderSpecNAStrategy, CategoricalNAStrategy

## Start a local dask cluster

In [ ]:
sess = Session('local')

In [ ]:
sess

In [ ]:
?Session

## Define relevant functions

In [ ]:
def make_v16_config(*, ancestry, ctl_ratio=19, time_model=ModelType.STANDARD, feature_set='67bm', feature_selection_iterations=None):
    """Configuration generator for the different predefined scenarios.
    
    ancestry (obj): Takes path to file containing relevant sample IDs
    ctrl_ratio (int): Ratio of controls to cases (also used to determine number of iterations for olink feature selection). 19 for EUR ancestry and 67 biomarkers, 9 for all other ancestries/feature-sets
    time_model (obj): Time-model to subset cases according to time lag between diagnosis and sample collection. Takes one of the following {ModelType.STANDARD, ModelType.PROGNOSTIC, ModelType.DIAGNOSTIC} for time-agnostic, prognostic and diagnostic time-models, respectively.
    feature_set (str): Pre-implemented feature set to run MILTON on. Takes one of {'67bm', 'olink-only', 'olink-and-67bm'}
    feature_selection_iterations: Number of iterations to perform Boruta feature selection for. In each iteration a different age-sex-number matched control set is sampled. Union of confirmed (or tentative, if selected) features across all iterations are used for further training XGBoost classifier. 
                                    - Unless otherwise stated, ctl_ratio is used as number of iterations for feature_selection.
                                    - Set to 0 if no feature pre-selection is desired.
                                    - No feature selection is performed for 67 biomarkers.
                                    - To preserve any co-variates or features, use the conf().feature_selection.preserved option as shown below
                                    
    """
    
    conf = Settings()

    if ancestry:
        # use specific settings, not the defaults
        qv_model_dir, qv_subject_subset = ancestry
        conf().analysis.qv_model_dir = qv_model_dir
        conf().analysis.qv_subject_subset = qv_subject_subset
        # specific ancestry means training is also constrained
        ids = pd.read_csv(qv_subject_subset, usecols=['eid'])['eid']
        conf().patients.used_subjects = ids.to_list()
        
    if ctl_ratio:
        conf().patients.controls_to_cases_ratio = ctl_ratio
        
    if feature_selection_iterations is None:
        feature_selection_iterations=ctl_ratio
    
    if feature_set=='67bm': #67 biomarkers only
        conf.features.biomarkers = True
        conf.features.respiratory=True
        conf.features.overall_health=True
        conf.features.olink = False
        conf.features.olink_covariates = False
    
    elif feature_set=='olink-only': #olink proteomics only
        conf.features.biomarkers = False
        conf.features.olink = True
        conf.features.olink_covariates = True

        conf().feature_selection.iterations = feature_selection_iterations
        conf().feature_selection.preserved = [  # all covariates
            Col.AGE, 
            Col.GENDER, 
            'Alcohol intake frequency.',
            'Illnesses of father',
            'Illnesses of mother',
            'Smoking status',
            'Blood-type haplotype',
            'Body mass index (BMI)'
        ]
    
    elif feature_set=='olink-and-67bm': #olink and 67 bm
        conf.features.biomarkers = True
        conf.features.respiratory=True
        conf.features.overall_health=True
        conf.features.olink = True
        conf.features.olink_covariates = True

        conf().feature_selection.iterations = feature_selection_iterations #number of iterations to do feature selection for
        conf().feature_selection.preserved = [  # all covariates
            Col.AGE, 
            Col.GENDER, 
            'Alcohol intake frequency.',
            'Illnesses of father',
            'Illnesses of mother',
            'Smoking status',
            'Blood-type haplotype',
            'Body mass index (BMI)'
        ]
        #also don't do feature selection of these 67 traits, just the olink proteins
        ukb_biomarkers = DD.predefined(biomarkers=True, respiratory=True, overall_health=True)\
        .index.drop_duplicates()\
        .drop([Col.GENDER, Col.AGE])\
        .to_list()
        conf().feature_selection.preserved.extend(ukb_biomarkers)
    
    else:
        print('Feature set not defined. Proceeding with 67 biomarkers..')
        conf.features.biomarkers = True
        conf.features.respiratory=True
        conf.features.overall_health=True
        conf.features.olink = False
        conf.features.olink_covariates = False
        
    
    conf().preproc.na_imputation = 'median'
    conf().preproc.na_imputation_extra = {
        'Testosterone': GenderSpecNAStrategy(males='median', females='median'),
        Col.RHEUMATOID_FACTOR: ('constant', 0.0),
        Col.OESTRADIOL: GenderSpecNAStrategy(males=36.71, females=110.13),
    }

    conf().analysis.default_model = 'xgb'
    conf().analysis.hyper_parameters = {
        'n_estimators': [50, 100, 200, 300],
    }
    conf().analysis.hyper_param_metric = 'roc_auc' 
    conf().analysis.n_replicas = 10 #number of replicas to train XGBoost for. Different control sets (ctl_ratio x #cases) will be sampled per replica
    conf().analysis.evaluate_all_replicas = True
    
    conf().analysis.model_type = time_model
    return conf

## Call function with relevant parameters

In [ ]:
ctl_ratio=9
time_model='time_agnostic'
ancestry_name='EUR' #one of AFR, AMR, EAS, EUR, SAS 
feature_set='olink-and-67bm'
#specify which ICD10 code to sample cases and controls from
code='I871'
out_dir=os.path.join('.', 'results', code, ancestry_name, feature_set, time_model)

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

if time_model=='time_agnostic':
    timemodel=ModelType.STANDARD
elif time_model=='prognostic':
    timemodel=ModelType.PROGNOSTIC
elif time_model=='diagnostic':
    timemodel=ModelType.DIAGNOSTIC
else:
    print('time_model not defined')

#call config function with relevant parameters
settings = make_v16_config(
    ancestry=(<path_to_sample_IDs_with_WGS_data_belonging_to_specified_ancestry>),
    ctl_ratio=ctl_ratio,
    time_model=timemodel,
    feature_set=feature_set
    
)

#specify which ICD10 code to use for cases and controls
settings().patients.spec = ICD10.find_by_code(code)

settings().analysis.min_cases = 0

In [ ]:
settings()

Feel free to explore settings() object to look for more options that you think might require tweeking. Below I have captured the most common use cases.

Please note that when specifying your own case/control ids, MILTON doesn't know which ICD10 to use for time-lag calculation. Therefore, only time-agnostic model is implemented in this case. Please perform the subsetting yourself while deriving case and control ids.

**Cases and controls**

- To specify multiple ICD10 codes

```
desired_codes=['N18', 'C50', 'C61', 'F30']
all_codes_list=[]
for code in desired_codes:
    all_codes_list.append(list(ICD10.find_by_code(code)))
    
settings().patients.spec = list(itertools.chain(*all_codes_list))
```

- To specify your own list of cases and controls:

    - using case ids only
    
    ```settings().patients.spec = pd.Series(1, index=case_ids)```

    - using case and control ids
    
    ```settings().patients.spec = pd.concat([pd.Series(1, index=case_ids), pd.Series(0, index=control_ids)])```


- To set minimum number of training cases to 0, default 100

```settings().analysis.min_cases = 0```

- To specify control subset for training XGBoost

```settings().patients.training_controls = <list of control ids>```

- To specify control subset for performing collapsing analysis

```settings().patients.collapsing_controls = <list of control ids>```

- To remove certain subjects from analysis.

```settings().patients.used_subjects = list(set(settings().patients.used_subjects).difference(<list of subject ids to exclude>)```
    
- To perform low power collapsing analysis (only on subjects with Olink or NMR metabolomics data, for example, and not the entire UKB cohort)

```settings().analysis.collapsing_on_data_index = True```


**Features**

- To run MILTON on a subset of proteins:

```settings().features.olink = <list of olink protein names such as ['C2', 'TNF']>```

- To add extra features from UKB based on their field ids:

```settings().features.ukb_custom = [21025, 21027] #UKB field IDs for additional 7 features```

**Custom feature imputation**

```
settings().preproc.na_imputation_extra = {
            'Testosterone': GenderSpecNAStrategy(males='median', females='median'),
            Col.RHEUMATOID_FACTOR: ('constant', 0.0),
            Col.OESTRADIOL: GenderSpecNAStrategy(males=36.71, females=110.13),
            'Had menopause': CategoricalNAStrategy(),
            'Smoking status': CategoricalNAStrategy(),
            'Age when periods started (menarche)': ('constant', -5), 
            'Age at first live birth': ('constant', -5),
        }
```

## Run MILTON and save results to file

In [ ]:
#run Evaluator and save report
ev = Evaluator(settings)
ev.run()
ev.save_report(out_dir)